# Lendo dados

In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

df = pd.read_csv("dataset/games.csv")

print(df.columns)

Index(['Game Title', 'User Rating', 'Age Group Targeted', 'Price', 'Platform',
       'Requires Special Device', 'Developer', 'Publisher', 'Release Year',
       'Genre', 'Multiplayer', 'Game Length (Hours)', 'Graphics Quality',
       'Soundtrack Quality', 'Story Quality', 'User Review Text', 'Game Mode',
       'Min Number of Players'],
      dtype='object')


# Pre processamento dos dados

## Limpeza dos dados

- Remover colunas irrelevantes

In [19]:
cols_to_remove = [
    'Graphics Quality',     
    'Soundtrack Quality',
    'Story Quality',
    'User Review Text',      
]

df.drop(inplace=True, columns=cols_to_remove)
df.head(8)

,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,Offline,1
5,Grand Theft Auto V,38.6,Adults,51.73,Xbox,No,Capcom,Capcom,2017,RPG,Yes,48.8,Offline,4
6,Sid Meier’s Civilization VI,33.1,Adults,46.44,Mobile,No,Game Freak,Innersloth,2020,Simulation,No,36.9,Online,3
7,Just Dance 2024,32.3,Teens,36.92,Nintendo Switch,No,Capcom,Take-Two Interactive,2012,Strategy,No,52.1,Offline,3


- Remover colunas irrelevantes

In [20]:
before = len(df)
df.dropna(inplace=True)
after = len(df)

print(f"Linhas antes: {before} | depois: {after}")

df.head(8)

Linhas antes: 47774 | depois: 47774


,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,Offline,1
5,Grand Theft Auto V,38.6,Adults,51.73,Xbox,No,Capcom,Capcom,2017,RPG,Yes,48.8,Offline,4
6,Sid Meier’s Civilization VI,33.1,Adults,46.44,Mobile,No,Game Freak,Innersloth,2020,Simulation,No,36.9,Online,3
7,Just Dance 2024,32.3,Teens,36.92,Nintendo Switch,No,Capcom,Take-Two Interactive,2012,Strategy,No,52.1,Offline,3


- Remover linhas duplicadas

In [21]:
before = len(df)
df.drop_duplicates(inplace=True)
afeter = len(df)

print(f"Linhas antes: {before} | depois: {after}")
df.head(8)

Linhas antes: 47774 | depois: 47774


,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,Offline,1
5,Grand Theft Auto V,38.6,Adults,51.73,Xbox,No,Capcom,Capcom,2017,RPG,Yes,48.8,Offline,4
6,Sid Meier’s Civilization VI,33.1,Adults,46.44,Mobile,No,Game Freak,Innersloth,2020,Simulation,No,36.9,Online,3
7,Just Dance 2024,32.3,Teens,36.92,Nintendo Switch,No,Capcom,Take-Two Interactive,2012,Strategy,No,52.1,Offline,3
